# Query Generation from Natural Language Descriptions
This project seeks to investigate the feasibility of automating the generation of SQL queries using open-source LLMs. By fine-tuning these models to interpret natural language instructions and translate them into accurate SQL queries, we can enable domain experts and non-technical stakeholders to interact with data in a more intuitive and effective manner.

This project will focus on fine-tuning open-source LLMs for query generation. The following sections will give an overview of the objectives, scope, and expected results.

## Resources

### Dataset
Dataset to generate queries on : http://archive.ics.uci.edu/dataset/852/gender+gap+in+spanish+wp

### Base Model
We can choose any open-source LLM as base model to finetune.


### Data Store
You have to load the dataset over Bigquery platform and your must write code to connect to this datastore to run the bigquery queries.
You are advised to explore Bigquery platform using the GCP free-tier environment before attempting the assignment.

### Compute Resources
You can use any platform to finetune the model. I will be using Unsloth in this project.


## Important Instructions

*   We can upload the model we train over drive.
*   I will be including all information/code/schema/steps to reproduce finetuning process and run the test cases.


In [ ]:
# Requirements
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

### Use the following sections as a guide to organize your code, you can change the structure when needed

Link to the Dataset used to finetune model: https://huggingface.co/datasets/b-mc2/sql-create-context

Prepare and Load Finetuning Data

In [ ]:
prompt = """You are a SQL query gereration assistant, below is an instruction that describes a task, paired with an input that provides the details of the variables for further context. Write a response that completes the request step by step.

### Question:
{}

### Context:
{}

### Answer:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["question"]
    inputs       = examples["context"]
    outputs      = examples["answer"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input, output)
        texts.append(text)
    return { "text" : texts, }
pass

dataset = load_dataset("b-mc2/sql-create-context", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

Load Base Model

I have used Mistral 7b model (4-bit version)

In [ ]:
model_name = "unsloth/mistral-7b-bnb-4bit" # Can choose from many open source models, check Unsloth's Hugging Face for more info
max_seq_length = 4096 # Can increase or decrease it based on your liking or use case
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Used 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla V100-SXM2-16GB. Max memory: 15.773 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # supports rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Post-Processing on Model

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

Step,Training Loss
1,2.448200
2,2.448000
3,2.359900
4,2.131300
5,1.524600
6,1.212300
7,0.773200
8,0.854200
9,0.748500
10,0.766300


## Saving the Fine Tuned Model

For some reason the authentication is failing for my google account and provide the link to the model. Therefore, I am unable to upload the model to my google drive. However, I have given another code that saves the model in the colab environment from which the files can be downloaded.

In [ ]:
# from google.colab import drive
# from google.colab import auth
# auth.authenticate_user()

# # Mount your Google Drive
# drive.mount('/content/gdrive')

# saved_model_path = "mistral_7b_finetuned"

# # Specify the destination path in your Google Drive
# destination_path = "/content/gdrive/My Drive/models/mistral_7b_finetuned"

# # Upload the model
# import shutil
# shutil.copy2(saved_model_path, destination_path)

In [ ]:
trainer.save_model("mistral_7b_finetuned")

Bigquery Table Creation & Bigquery connection creation

In [ ]:
from google.cloud import bigquery
from google.colab import auth

auth.authenticate_user()

# Set project, dataset, and table IDs
project_id = "zscalar"
dataset_id = "zdata"
table_id = "ztable"

# Create a BigQuery client object
client = bigquery.Client(project=project_id)

In [ ]:
# # Define the table schema
# schema = [
#     bigquery.SchemaField("gender", "INTEGER"),
#     bigquery.SchemaField("C_api", "INTEGER"),
#     bigquery.SchemaField("C_man", "INTEGER"),
#     bigquery.SchemaField("E_NEds", "INTEGER"),
#     bigquery.SchemaField("E_Bpag", "INTEGER"),
#     bigquery.SchemaField("firstDay", "INTEGER"),
#     bigquery.SchemaField("lastDay", "INTEGER"),
#     bigquery.SchemaField("NEds", "INTEGER"),
#     bigquery.SchemaField("NDays", "INTEGER"),
#     bigquery.SchemaField("NActDays", "INTEGER"),
#     bigquery.SchemaField("NPages", "INTEGER"),
#     bigquery.SchemaField("NPcreated", "INTEGER"),
#     bigquery.SchemaField("pagesWomen", "INTEGER"),
#     bigquery.SchemaField("wikiprojWomen", "INTEGER"),
#     bigquery.SchemaField("ns_user", "INTEGER"),
#     bigquery.SchemaField("ns_wikipedia", "INTEGER"),
#     bigquery.SchemaField("ns_talk", "INTEGER"),
#     bigquery.SchemaField("ns_userTalk", "INTEGER"),
#     bigquery.SchemaField("ns_content", "INTEGER"),
#     bigquery.SchemaField("weightIJ", "FLOAT"),
#     bigquery.SchemaField("NIJ", "INTEGER"),
# ]

# # Construct a table reference object
# table_ref = client.dataset(dataset_id).table(table_id)

# # Create the table
# table = bigquery.Table(table_ref, schema=schema)
# table = client.create_table(table)

#### Tests
Lets test the model on the below promps and generate  answer from your model for these queries.



In [ ]:
"""
1. Give me distinct values of I index of stratum in the entire database.
2. Return count of rows who have maximum number of total editions.
3. Tell me the number of editions published by female editors which have been active for atleast a year and have more than 1 edit in namespace user talk.
4. Tell me the total number of editions published by editors which have identified as female by wikimedia api.
5. Show the average number of active days for editors who have done edits in pages related to women.
6. Return average weight for editors who have created more than hundred pages and grouped over gender extracted from content coding column along with count for each group and sorted in ascending order of average weight.
7. Give query to get the average weight for editors who have created more than hundred pages and grouped over gender extracted from content coding column along with count for each group and sorted in ascending order of average weight.
8. Return maximum number of edits in namespace user, average number of edits in namespace talk and average number of elements in stratum IJ in the database grouped over I index of stratum IJ and sorted in ascending order of I index of stratum IJ for editors having having correcting weight for stratum IJ more than .5 & atleast 50 number of edits in WikiProjects related to women & with unknown gender from both wikimedia & content coding.
"""

'\n1. Give me distinct values of I index of stratum in the entire database.\n2. Return count of rows who have maximum number of total editions.\n3. Tell me the number of editions published by female editors which have been active for atleast a year and have more than 1 edit in namespace user talk.\n4. Tell me the total number of editions published by editors which have identified as female by wikimedia api.\n5. Show the average number of active days for editors who have done edits in pages related to women.\n6. Return average weight for editors who have created more than hundred pages and grouped over gender extracted from content coding column along with count for each group and sorted in ascending order of average weight.\n7. Give query to get the average weight for editors who have created more than hundred pages and grouped over gender extracted from content coding column along with count for each group and sorted in ascending order of average weight.\n8. Return maximum number of e

In [ ]:
questions = [
  "Give me distinct values of I index of stratum in the entire database.",
  "Return count of rows who have maximum number of total editions.",
  "Tell me the number of editions published by female editors which have been active for atleast a year and have more than 1 edit in namespace user talk.",
  "Tell me the total number of editions published by editors which have identified as female by wikimedia api.",
  "Show the average number of active days for editors who have done edits in pages related to women.",
  "Return average weight for editors who have created more than hundred pages and grouped over gender extracted from content coding column along with count for each group and sorted in ascending order of average weight.",
  "Give query to get the average weight for editors who have created more than hundred pages and grouped over gender extracted from content coding column along with count for each group and sorted in ascending order of average weight.",
  "Return maximum number of edits in namespace user, average number of edits in namespace talk and average number of elements in stratum IJ in the database grouped over I index of stratum IJ and sorted in ascending order of I index of stratum IJ for editors having correcting weight for stratum IJ more than .5 & atleast 50 number of edits in WikiProjects related to women & with unknown gender from both wikimedia & content coding."
]

answer = ["" for _ in range(8)]  # Initialize an empty list to store the answers

In [ ]:
def generate_response(instruction):
  inputs = tokenizer(
  [
      prompt.format(
          instruction, # instruction
          """CREATE TABLE ztable (gender TINYINT UNSIGNED, -- (0: unknown, 1: male, 2: female)
            C_api VARCHAR(10), -- Gender from WikiMedia API (female/male/unknown)
            C_man TINYINT UNSIGNED, -- Gender from content coding (1: male, 2: female, 3: unknown)
            E_NEds TINYINT UNSIGNED, -- I index of stratum IJ (0,1,2,3)
            E_Bpag TINYINT UNSIGNED, -- J INDEX of stratum IJ (0,1,2,3)
            firstDay DATETIME, -- First edition (YYYYMMDDHHMMSS)
            lastDay DATETIME, -- Last edition (YYYYMMDDHHMMSS)
            NEds INT UNSIGNED, -- Total number of edits
            NDays INT UNSIGNED, -- Number of days (lastDay-firstDay+1)
            NActDays INT UNSIGNED, -- Number of days with edits
            NPages INT UNSIGNED, -- Number of different pages edited
            NPcreated INT UNSIGNED, -- Number of pages created
            pagesWomen INT UNSIGNED, -- Edits in pages related to women
            wikiprojWomen INT UNSIGNED, -- Edits in WikiProjects related to women
            ns_user INT UNSIGNED, -- Edits in namespace user
            ns_wikipedia INT UNSIGNED, -- Edits in namespace wikipedia
            ns_talk INT UNSIGNED, -- Edits in namespace talk
            ns_userTalk INT UNSIGNED, -- Edits in namespace user talk
            ns_content INT UNSIGNED, -- Edits in content pages
            weightIJ DECIMAL(2,6), -- Correcting weight for stratum IJ
            NIJ INT UNSIGNED -- Number of elements in stratum IJ);""", # input
          "", # output
      )
  ]*1, return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
  decoded_output = tokenizer.batch_decode(outputs)

  answer_start = decoded_output[0].find("Answer:\n")  # Find the starting position of the answer section
  if answer_start != -1:  # Check if "Answer:" is found
    answer_end = decoded_output[0].find("\n", answer_start + 8)  # Find the next newline after "Answer:\n"
    answer[i] = decoded_output[0][answer_start+len("Answer:\n"):answer_end]  # Extract the SQL query
    # return answer[i]  # return the SQL query
  else:
    return "No answer found."

In [ ]:
for i in range(8):
  generate_response(questions[i])

In [ ]:
for i in range(8):
  print(answer[i])

SELECT DISTINCT E_NEds FROM editors
SELECT COUNT(*) FROM editors WHERE NEds = (SELECT MAX(NEds) FROM editors)
SELECT COUNT(*) FROM editors WHERE gender = 2 AND C_api = "female" AND C_man = 2 AND E_NEds = 0 AND E_Bpag = 0 AND firstDay < lastDay AND NEds > 1 AND ns_user > 0 AND ns_wikipedia > 0 AND ns_talk > 0 AND ns_userTalk > 0 AND ns_content > 0 AND weightIJ > 0 AND NIJ > 0</s
SELECT COUNT(*) FROM editors WHERE C_api = "female" AND E_NEds = 1
SELECT AVG(NActDays) FROM editors WHERE pagesWomen > 0 AND gender = 2 AND C_api = "female"
SELECT AVG(weightIJ) FROM editors WHERE NPcreated > 100 AND C_man = 1 GROUP BY gender ORDER BY AVG(weightIJ) ASC
SELECT AVG(weightIJ) FROM editors WHERE NPcreated > 100 AND C_man = 1 GROUP BY gender ORDER BY AVG(weightIJ) ASC
SELECT MAX(ns_user) FROM editors WHERE ns_wikipedia = 1 AND ns_talk = 1 AND ns_content = 1 AND ns_userTalk = 1 AND weightIJ > .5 AND pagesWomen >= 50 AND wikiprojWomen = 1 AND gender = 2 AND C_api = "female" AND C_man = 2 AND NEds >= 5

In [ ]:
rows = []
for i in range(8):
  rows = client.query(answer[i])
  for row in rows.result():
    print(row)

## Future Tasks

## Regression Using LLM

Apart from returning the bigquery query for any textual natural language prompt, we want the LLM to also be able to predict the "weightIJ" given the parameters in textual format(Ignore the first day and last day atributes for this task).

For Example, for the query

"What will be the weightIJ for an editor with following attributes:
gender: male,
C_api: 1,
C_man: 1,
E_NEDs: 2,
NEDs: 300,
NDays: 500,
NActDays: 300,
NPcreated: 50,
pagesWomen: 10,
wikiprojWomen: 5,
ns_user: 0,
ns_wikipedia: 3,
ns_talk: 1
ns_userTalk: 2,
ns_content: 3,
NIJ: 100
"
<br>
Please Fine-tune the language model trained above for this task as well and provide all the steps along with code in structured manner as exemplified above.

Unfortunately, I was not able to find a sufficiently large dataset that the model could be finetuned on. Only providing a few shot examples would not be enough to train the model. This issue could be resolved by generating synthetic dataset. Then the rest of the steps would be the same as performed above with a slight change in the prompt given to the model.

## Model Evaluation
How would you evaluate your model so that you know General Availability would be successful?

My plan for evaluating the fine-tuned LLM for BigQuery query generation to determine its readiness for general availability:

1. Accuracy: Determine the percentage of queries generated by the model that are syntactically correct and semantically identical to the expected outcome on a validation set created independently of the original dataset. This can be done manually by domain experts or by automated test suites that compare the model's output to pre-written SQL queries for the same prompts.

2. Generalizability: Test the model's performance on unseen data using a dataset with different distributions of questions and contexts than the training data. This ensures that the model does not overfit the training data and can manage variances in natural language instructions.

3. Error Analysis: Examine the model's errors (e.g., syntax mistakes, erroneous joins, inaccurate aggregations). This helps highlight areas for improvement, such as retraining the model with more relevant data or enhancing the training process.

4. A/B Testing: If possible, test the LLM alongside a regular interface for writing BigQuery queries with a small number of users. Determine which interface consumers prefer and how their success rates differ between the two interfaces.
